# **Clustering Top 5 football players**

### Les imports

In [18]:
import requests
from bs4 import BeautifulSoup
import csv
import os
import time

### Récuparation de tous les liens des équipes du top 5 européen

In [19]:
class TeamScraper:
    def __init__(self, url):
        self.url = url

    def scrape_teams(self):
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, 'html.parser')
        team_table = soup.find('table', {'id': 'big5_table'})
        teams_data = []

        for row in team_table.find('tbody').find_all('tr'):
            team_cell = row.find('td', {'data-stat': 'team'})
            if team_cell:
                team_name = team_cell.get_text(strip=True)
                team_link = 'https://fbref.com' + team_cell.find('a')['href'] + '/'
                teams_data.append([team_name, team_link])

        # Save team data to CSV
        with open('../data/teams_data.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Team Name', 'Team URL'])
            writer.writerows(teams_data)

        print("Teams data saved to teams_data.csv")

In [20]:
team_scraper = TeamScraper('https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats')
team_scraper.scrape_teams()

Teams data saved to teams_data.csv


### Récuparation de tous les liens des joueurs de chaque équipe

In [49]:
class PlayerScraper:
    def __init__(self, teams_file):
        self.teams_file = teams_file

    def scrape_players(self):
        # Open the output CSV file in append mode (use mode='a')
        output_file = '../data/players_data.csv'
        
        # Open CSV file once outside the player loop
        with open(output_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Team Name', 'Player Name', 'Position', 'Player URL'])  # Write header once

            # Open the teams file and iterate through teams
            with open(self.teams_file, newline='') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                
                # Loop through each team
                for row in reader:
                    team_name, team_url = row
                    print(f"Scraping players from {team_name}...")

                    # Fetch the page content and parse it
                    response = requests.get(team_url)
                    soup = BeautifulSoup(response.content, 'html.parser')

                    # Find the player table
                    player_table = soup.find('table', {'class': 'stats_table sortable min_width'})

                    # Iterate through all player rows
                    for player_row in player_table.find('tbody').find_all('tr'):
                        print(player_row)  # For debugging purposes
                        
                        # Extract player details
                        player_name = player_row.find('td', {'data-stat': 'player'}).get_text(strip=True)
                        player_position = player_row.find('td', {'data-stat': 'position'}).get_text(strip=True)
                        player_link = 'https://fbref.com' + player_row.find('td', {'data-stat': 'player'}).find('a')['href'] + '/'
                        
                        # Write player data to the CSV file immediately
                        writer.writerow([team_name, player_name, player_position, player_link])

                    # Add a delay after each request to avoid being blocked (5 requests per minute = 12 seconds delay)
                    time.sleep(12)

        print(f"Players data saved to {output_file}")

In [51]:
player_scraper = PlayerScraper('../data/teams_data.csv')
player_scraper.scrape_players()

Scraping players from Monaco...


AttributeError: 'NoneType' object has no attribute 'find'